# Анализ данных датасета Мегафона

28.03.2023


Задание предусматривает исследование датасетов с клиентами и их трафиком компании "Мегафон" для нахождения интересных особенностей и закономерностей и выделении 10 наиболее перспектиных городов с точки зрения развития туристической отрасли.

Есть данные, хранящиеся в файлах `Tourists_CITY_2022.csv` с данными об абонентах счетах из внутренних источников компании за период с *01.01.2021* по *01.12.2022*.

# Описание задания

* `Snap_date` - дата начала месяца туризма
* `Id_hash` - id абонента. Один абонент может встречаться несколько раз в разных месяцах или городах
* `days` - продолжительность присутствия в днях в точке туризма 
* `home_region` - номер домашнего региона 
* `city` - точка туризма 
* `gender` - предсказанный моделью пол абонента. 
> * 1 – мужчина;
> * 2 – женщина;
> * 3 – неизвестно;
* `age` - предсказанный моделью возраст абонента в годах 
* `mou` - голосовой трафик абонента за месяц (snap_date) в минутах 
* `mou_out` - исходящий голосовой трафик абонента за месяц в минутах 
* `dou` - интернет-трафик абонента за месяц, МБ
* `Interests` - список интересов абонента за месяц snap_date 
* `top_service` - url, по которому у абонента больше всего трафика за месяц 
* `volume_sum` - трафик абонента за время присутствия в точке туризма, МБ
* `dl/ul_volume_sum` - разложение volume_sum на uplink, downlink, МБ
* `Column_1` - служебное поле, не информативное

## Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mth
import calendar

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import datetime as dt
from datetime import datetime, timedelta

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('display.max_columns', None)

## Чтение файлов

In [ ]:
path_cloud = '/kaggle/input/spgu-olimp/Олимпиада для первых_2023 материалы/Кейс, критерии и данные для кейса/Данные_2021/'


In [ ]:
path_local = '/kaggle/input/spgu-olimp/Олимпиада для первых_2023 материалы/Кейс, критерии и данные для кейса/Данные_2021/'
csv_files = ['Tourists_Абинск_2021.csv', 'Tourists_Александров_2021.csv', 'Tourists_Армавир_2021.csv',
             'Tourists_Байкальск_2021.csv', 'Tourists_Белореченск_2021.csv', 'Tourists_Вязники_2021.csv', 
             'Tourists_Горно-Алтайск_2021.csv', 'Tourists_Городищи_2021.csv', 'Tourists_Гороховец_2021.csv',
             'Tourists_Горячий Ключ_2021.csv', 'Tourists_Гулькевичи_2021.csv', 'Tourists_Ейск_2021.csv', 
             'Tourists_Ковров_2021.csv', 'Tourists_Кольчугино_2021.csv', 'Tourists_Кореновск_2021.csv', 
             'Tourists_Кропоткин_2021.csv', 'Tourists_Крымск_2021.csv', 'Tourists_Курганинск_2021.csv',
             'Tourists_Лабинск_2021.csv', 'Tourists_Лакинск_2021.csv', 'Tourists_Меленки_2021.csv', 
             'Tourists_Муром_2021.csv', 'Tourists_Новокубанск_2021.csv', 'Tourists_Петушки_2021.csv',
             'Tourists_Покров_2021.csv', 'Tourists_Сириус_2021.csv', 'Tourists_Славянск-на-Кубани_2021.csv', 
             'Tourists_Сортавала_2021.csv', 'Tourists_Струнино_2021.csv', 'Tourists_Темрюк_2021.csv', 
             'Tourists_Тимашевск_2021.csv', 'Tourists_Тихорецк_2021.csv', 'Tourists_Туапсе_2021.csv', 
             'Tourists_Усть-Лабинск_2021.csv', 'Tourists_Хадыженск_2021.csv', 'Tourists_Юрьев-Польский_2021.csv']

df_2021 = pd.read_csv(path_local + 'Tourists_Абинск_2021.csv')
for file in csv_files:
    if file != 'Tourists_Абинск_2021.csv':
        df = pd.read_csv(f'{path_local}{file}')
        df_2021 = pd.concat([df_2021, df], ignore_index=True)

In [ ]:
path_local = '/kaggle/input/spgu-olimp/Олимпиада для первых_2023 материалы/Кейс, критерии и данные для кейса/Данные_2022/'
csv_files = ['Tourists_Абинск_2022.csv', 'Tourists_Александров_2022.csv', 'Tourists_Армавир_2022.csv',
             'Tourists_Байкальск_2022.csv', 'Tourists_Белореченск_2022.csv', 'Tourists_Вязники_2022.csv',
             'Tourists_Горно-Алтайск_2022.csv', 'Tourists_Городищи_2022.csv', 'Tourists_Гороховец_2022.csv',
             'Tourists_Горячий Ключ_2022.csv', 'Tourists_Гулькевичи_2022.csv', 'Tourists_Ейск_2022.csv',
             'Tourists_Ковров_2022.csv', 'Tourists_Кольчугино_2022.csv', 'Tourists_Кореновск_2022.csv',
             'Tourists_Кропоткин_2022.csv', 'Tourists_Крымск_2022.csv', 'Tourists_Курганинск_2022.csv',
             'Tourists_Лабинск_2022.csv', 'Tourists_Лакинск_2022.csv', 'Tourists_Меленки_2022.csv',
             'Tourists_Муром_2022.csv', 'Tourists_Новокубанск_2022.csv', 'Tourists_Петушки_2022.csv',
             'Tourists_Покров_2022.csv', 'Tourists_Сириус_2022.csv', 'Tourists_Славянск-на-Кубани_2022.csv',
             'Tourists_Сортавала_2022.csv', 'Tourists_Струнино_2022.csv', 'Tourists_Темрюк_2022.csv',
             'Tourists_Тимашевск_2022.csv', 'Tourists_Тихорецк_2022.csv', 'Tourists_Туапсе_2022.csv',
             'Tourists_Усть-Лабинск_2022.csv', 'Tourists_Хадыженск_2022.csv', 'Tourists_Юрьев-Польский_2022.csv']

df_2022 = pd.read_csv(path_local + 'Tourists_Абинск_2022.csv')
for file in csv_files:
    if file != 'Tourists_Абинск_2022.csv':
        df = pd.read_csv(f'{path_local}{file}')
        df_2022 = pd.concat([df_2022, df], ignore_index=True)

## Вывод таблиц

### 2021 год

In [ ]:
df_2021.sample(10, random_state=7)

In [ ]:
prev = df_2021.shape[0]
df_2021.info()

В объединенном датасете за 2021 год всего __3.8 млн строк__. Удалим в дальнейшем столбец `Unnamed: 0` за ненадобностью пользования. Также формат `snap_date` не совпадает с datetime. 

In [ ]:
df_2021.isna().sum().sort_values(ascending=False)

In [ ]:
# Пропуски в % соотношении общего кол-ва строк в дф

df_2021.isnull().sum().sort_values(ascending=False)/df_2021.shape[0]*100

__591 тыс.__ строк с интернет-трафиком абонента за месяц за 2021 год или __15.2%__ пропущено. Это может быть связано как с технической ошибкой, так и с натуральной природой пропуска, то есть пользователь интернетом не пользовался. То же самое касается полей с голосовым трафиком абонентов, где __11.05%__ и __4.2%__ пропусков – общий голосовй трафик и исходящий, соответственно. 

In [ ]:
df_2021.describe()

Сразу заметим аномальные отрицательные значения в полях с возрастом и регионом. 

In [ ]:
df_2021.duplicated().sum()

Явные дубликаты отсутствуют.

### 2022 год

In [ ]:
df_2022.sample(10, random_state=7)

Сразу отметим, что в данных за 2022 год присутствуют дополнительные поля: интересы, самый популярный сайт и 3 характеристики потребления интернет-трафика. 

In [ ]:
prev = df_2022.shape[0]
df_2022.info()

Данных за 2022 год также **3.8 млн. строк.**
`snap_date` не соответсвует нужному типу данных. 

In [ ]:
df_2022.isna().sum().sort_values(ascending=False)

In [ ]:
# Пропуски в % соотношении общего кол-ва строк в дф

df_2022.isnull().sum().sort_values(ascending=False)/df_2022.shape[0]*100

Касательно пропусков, то тут ситуация следующая: также, как и за 2021 год не хватает __14.44%__, __11.66%__ и __4.68%__ или __551 тыс.__, __445 тыс.__ и __178 тыс.__ по интернет трафику и голосовому трафику. __Для 21.5%__ всех строк за 2022 год нет данных по __интересам пользователей__ и __15%__ по __самому популярному сервису__. И __15-16%__ данных отсутствует __по трафику абонента за время присутствия в точке туризма__ и __разложения volume_sum__ на uplink, downlink. 

In [ ]:
df_2022.describe()

* Опять отрицательные значения для "_домашнего региона_". 
* Есть нулевые значения для столбца с возрастом, что означает о провале предсказания возраста абонента.
* Есть абоненты, которые как и в 2021 году, _разговаривали по телефону_ более __25 тыс. минут за месяц__. или аномальные значения `volume_sum`, когда медианный показатель потребления трафика абонента в точке туризма равен __1.1 млрд.__ МБ или 1100 ТБ, что очевидно, не соответствует истине. 

In [ ]:
df_2022.duplicated().sum()

Явные дубликаты отсутствуют.

## **Вывод** 
Итого после предварительного просмотра данных у нас имеется:
* **49 543 строки** - всего в датафрейме `data`;
* **1 из 8 столбцов** - имеют пропуски в датафрейме `data`;
* **0.46%** – пропусков в столбце `Ставка тарифа`;
* **356 846 строк** - всего в датафрейме `bills`;
* **1 из 5 столбцов** - имеют пропуски в датафрейме `bills`;
* **585 дубликатов** – в датафрейме `bills`;
* **24.98 у.е.** – средняя сумма платежа;
* **2006-08-30 - 2021-11-01** – временной отрезок таблицы `data`;
* **2018-01-04 - 2021-10-13** – временной отрезок таблицы `bills`;

# Предобработка данных

## Удаление и добавление столбцов

In [ ]:
df_2021 = df_2021.drop(columns='Unnamed: 0') # выкидываем ненужный столбец
df_2021['snap_date'] = pd.to_datetime(df_2021['snap_date']) # меняем формат

df_2021['month'] = df_2021['snap_date'].dt.month
df_2021['year'] = df_2021['snap_date'].dt.year
df_2021['month_name'] = df_2021['snap_date'].dt.strftime('%B')

In [ ]:
df_2022 = df_2022.drop(columns='Unnamed: 0') # выкидываем ненужный столбец
df_2022['snap_date'] = pd.to_datetime(df_2022['snap_date']) # меняем формат

df_2022['month'] = df_2022['snap_date'].dt.month
df_2022['year'] = df_2022['snap_date'].dt.year
df_2022['month_name'] = df_2022['snap_date'].dt.strftime('%B')

## Устранение аномалий

### 2021 год

Взглянем на диаграмму с распределением некоторых параметров таблиц.

#### Возраст

In [ ]:
def boxplot(field):
    plt.boxplot(df_2021[field], showcaps=True)

    # добавляем заголовок и метки осей
    plt.title(f'{field} Distribution')
    plt.ylabel(f'{field}')
    
    # выводим boxplot на экран
    plt.show()

In [ ]:
boxplot('age')

Есть точечные аномалии, когда модель ошиблась и предсказала возраст абонента неверно. Взглянем на общее кол-во таких абонентов, помня, что нулевое значение означает, что модель не может предсказать возраст, а затем заменим все аномальные значения на 0. 

In [ ]:
df_2021[df_2021['age'] < 0].shape[0]

In [ ]:
df_2021[df_2021['age'] == 0].shape[0]

In [ ]:
df_2021[df_2021['age'] > 99].shape[0]

In [ ]:
df_2021['age'] = df_2021['age'].apply(lambda x: 0 if x > 99 or x < 0 else x)

In [ ]:
boxplot('age')

Все же можем предположить, что удалять или отфильтровывать данные по возрастам нет смысла, так как туристом может быть человек любого возраста. 

In [ ]:
df_2021[['age']].describe()

In [ ]:
# 25-й перцентиль
def q25(x):
    return x.quantile(0.25)

# 75-й перцентиль
def q75(x):
    return x.quantile(0.75)

In [ ]:
def fences(field):
    q1 = df_2021[field].quantile(0.25)
    q3 = df_2021[field].quantile(0.75)
    iqr = q3 - q1
    upper_fence = q3 + 1.5 * iqr
    lower_fence = q1
    
    print(f'Граница максимальных значений {field}:', upper_fence)
    print(f'Граница минимальных значений {field}:', lower_fence)
    return upper_fence, lower_fence

In [ ]:
def distr(field, ul=1):
    if ul == 1:
        cities = df_2021.groupby('city').count()
        
        # Группируем исходный датафрейм по городам и считаем количество строк, где значение столбца field меньше границы максимальных значений
        df_filtered = df_2021[df_2021[field] < upper_fence].groupby('city').count()
        
        # Рассчитываем процентное соотношение количества строк с меньшим значением field к общему количеству строк для каждого города
        df_filtered['%'] = df_filtered['id_hash'] / cities['id_hash'] * 100
        
        # Создаем новый датафрейм с результатами
        df_result = pd.DataFrame({
            'city': cities.index,
            'count_less_uf': df_filtered['id_hash'],
            '%': df_filtered['%']
        })
        
        df_result = df_result.reset_index(drop=True)
        df_result = df_result.sort_values(by='%')
        return df_result
    else:
        cities = df_2021.groupby('city').count()
        
        # Группируем исходный датафрейм по городам и считаем количество строк, где значение столбца field меньше границы максимальных значений
        df_filtered = df_2021[df_2021[field] > lower_fence].groupby('city').count()
        
        # Рассчитываем процентное соотношение количества строк с меньшим значением field к общему количеству строк для каждого города
        df_filtered['%'] = df_filtered['id_hash'] / cities['id_hash'] * 100
        
        # Создаем новый датафрейм с результатами
        df_result = pd.DataFrame({
            'city': cities.index,
            'count_more_lf': df_filtered['id_hash'],
            '%': df_filtered['%']
        })
        
        df_result = df_result.reset_index(drop=True)
        df_result = df_result.sort_values(by='%')
        return df_result
        

In [ ]:
upper_fence, lower_fence = fences('age')

In [ ]:
distr('age')

In [ ]:
distr('age', 2)

Мы можем отфильтровать данные по верхней границе и не потерять чрезмерно большой объем данных. Более зрелые пользователи не представляют большой интерес, так как они менее охотно тратят средства на моб. связь, не так часто путешествуют, предпочитая домашнюю атмосферу, и в целом не являются показательными в плане пользования интернет и голосовыми услугами, так как их привычки кардинально отличаются от привычек более молодого поколения, к тому же с каждом годом кол-во таких абонентов сокращается ввиду ограниченности продолжительности жизни. 

Что же касается нижней границы, то мы не будем фильтровать абонентов до 30, так как они представляют большой интерес для компании в плане платежеспособности и лояльности. 

Проверим, сколько пользователей от общего числа, чей возраст определен не был.

In [ ]:
lower_fence = 0

distr('age', 2)

Менее 10% в каждом из городов, можем и отсечь. Но будет ли ситуация за 2022 год похожа? Можем создать отдельный дф для расчетов метрик по возрасту. 

In [ ]:
age_prev = df_2021.shape[0]

df_2021 = df_2021[df_2021['age'] < 65]

age_after = df_2021.shape[0]

data_lost = (age_prev - age_after) / prev * 100

def itogo(field):
    return print(f'Итого после предобработки данных мы потеряли {field:.2f}%')

itogo(data_lost)


In [ ]:
df_2021_no_age = df_2021[df_2021['age'] > 0]

In [ ]:
df_2021_no_age.groupby('city')['age'].agg(['mean', 'median', q25, q75, 'min', 'max']).sort_values(by=['mean', 'median'])

#### Количество проведенных дней

In [ ]:
days_prev = df_2021.shape[0]

In [ ]:
boxplot('days')

С днями все в порядке. Но по условию Ростислава, кол-во проведенных дней должно начинаться от 5. Взглянем, сколько это строк:

In [ ]:
df_2021[df_2021['days'] < 5].shape[0]

1.2 млн. строк мы должны удалить или все же оставить их для модели? Сколько в процентном соотношении занимают люди в каждом городе, которые были до 5 дней в точке.

In [ ]:
# Группируем данные по городам и количеству дней, проведенных в городе
grouped_data = df_2021.groupby(['city', pd.cut(df_2021['days'], [0, 4, float('inf')])]).size()

# Вычисляем общее количество строк для каждого города
total_rows = grouped_data.groupby('city').sum()

# Строим таблицу, где для каждого города показано количество строк, где турист провел менее 5 дней, в процентном соотношении
result = (grouped_data / total_rows * 100).unstack().fillna(0)

result = result.reset_index()
result.columns = ['City', 'Less than 5 days, %', 'More than 5 days included, %']
result = result.sort_values(by='More than 5 days included, %', ascending=False)
result

Выходит, что мы не можем не учитывать тех, кто провел менее 5 дней, так как в некоторых городах количество строк более половины от всех строк для города. 

Но мы можем сгруппировать дни по месяцам, таким образом оставив в таблице абонентов, которые находились в городе до 5 дней, но в том же месяце отправлялись в другой город из датасета. 

In [ ]:
less_5 = df_2021.pivot_table(index='id_hash', columns='month', values='days', aggfunc='sum', margins=True).sort_values(by='All', ascending=False)

In [ ]:
less_5 = less_5.query('All >= 5')
users = less_5.index

In [ ]:
df_2021 = df_2021[df_2021['id_hash'].isin(users)]

Теперь взглянем на максимальные и минимальные границы возможных значений, не попадабщих в категорию "выбросов":

In [ ]:
fences('days')

Но по рекомендации, следует учитывать до 20-22 дней включительно, поэтому поднимем планку и отфильтруем по значению 21 день в качестве верхней границы. Нижнюю границу учитывать не будем, так как мы ее уже учли.

In [ ]:
upper_fence, lower_fence = (22, 4)

In [ ]:
distr('days')

In [ ]:
distr('days', 2)

О чем нам говорит эта информация: 
- от _90.9% до 99.95_ абонентов 36 городов __проводят в городе до 22 дней__;
- от _50% до 95%_ абонентов 36 городов __проводят в городе от 5 дней__ (включительно); то есть есть города, такие как Тихорецк, Курганинск, Хадыженск, Сортавала, Белореченск и т.д., где 40-50% абонентов проводят в городе 1-2 дня. 

In [ ]:
df_2021 = df_2021[df_2021['days'] < 22]

days_after = df_2021.shape[0]

data_lost = (days_prev - days_after) / prev * 100 
itogo(data_lost)

In [ ]:
df_2021[['days']].describe()

#### Домашний регион

In [ ]:
df_2021.groupby('home_region').agg({'id_hash':'count'})

Более 14 тыс. записей. Взглянем на данные, когда указан "-1" регион. 

In [ ]:
df_2021[df_2021['home_region'] < 0].sample(10, random_state=10)

Какой-либо взаимосвязи не наблюдается. Отрицательные значения вероятно вызваны невозмодностью определения домашнего региона пользователя (что, по крайней мере, странно, ибо МФ должен знать наверняка – следовательно, вопрос к ментору). 

In [ ]:
home_2021 = df_2021.groupby('home_region').agg({'id_hash':'count'}).sort_values(by='id_hash', ascending=False)

In [ ]:
home_2021['%'] = home_2021['id_hash'] / home_2021['id_hash'].sum() * 100
home_2021

Более всего туристов из регионов в порядке убывания: 77, 52, 61, 78, 63 или в том же порядке Москва, Нижегородская обл., Ростовская обл., Санкт-Петербург и Самарская обл.

Менее всего туристов из регионов в порядке убывания: 79, 49, 4, 1, 87 или в том же порядке Еврейская автономная обл., Магаданская обл., Республика Алтай, Республика Адыгея, Чукотский автономный округ.

#### Туристический регион

In [ ]:
region_counts = df_2021['tourists_region'].value_counts()

# Вычисляем общее количество строк в датасете
total_rows = len(df_2021)

# Строим таблицу, где для каждого региона показано процентное соотношение количества абонентов от общего количества строк в датасете
result = pd.concat([region_counts, region_counts / total_rows * 100], axis=1)
result.columns = ['count', '%']

result

Аномалий нет, все нужные регионы присутствуют. Для удобства добавим столбец с наименованием регионов. 

In [ ]:
# Функция для возвращения наименования региона по коду
def get_region_name(region_code):
    region_names = {
        23: 'Краснодарский край',
        33: 'Владимирская обл.',
        4: 'Республика Алтай',
        38: 'Иркутская обл.',
        10: 'Республика Карелия'
    }
    return region_names.get(region_code)

# Добавляем новый столбец с наименованием региона
df_2021['tourist_region_name'] = df_2021['tourists_region'].apply(get_region_name)

#### Пол

In [ ]:
age_counts = df_2021['gender'].value_counts()

# Вычисляем общее количество строк в датасете
total_rows = len(df_2021)

# Строим таблицу, где для каждого региона показано процентное соотношение количества абонентов от общего количества строк в датасете
result = pd.concat([age_counts, age_counts / total_rows * 100], axis=1)
result.columns = ['count', '%']

result

Мужчин в датасете больше, чем женщин; по крайней мере из тех, что были определены верно. 10.6% абонентам пол установить не удалось. 

#### Голосовой трафик и голосовой исходящий трафик

In [ ]:
mou_prev = df_2021.shape[0]

Пропущенные значения означают, что абонент не звонил в течение месяца. Поэтому заменим пустышки на нули.

In [ ]:
df_2021.loc[df_2021['mou'].isnull(), 'mou'] = 0

In [ ]:
boxplot('mou')

In [ ]:
upper_fence, lower_fence = fences('mou')

In [ ]:
df_2021[['mou']].describe()

- *Среднее количество минут* голосового траифка равно 480 минутам в месяц.
- *Стандартное отклонение* значений равно 579 минутам в месяц, что является показателем наличия большого кол-ва выбросов относительно среднего;
- *Минимальное значение* равно 0.
- *25%*: первый квартиль, равный 73 минутам в месяц. Это означает, что 25% абонентов имеют среднее количество минут разговора в месяц, не превышающее 73 минут.
- *50%*: медиана значений, равная 296 минутам в месяц. Это означает, что 50% абонентов имеют среднее количество минут разговора в месяц, не превышающее 296 минут.
- *75%*: третий квартиль значений, равный 676 минутам в месяц. Это означает, что 75% абонентов имеют среднее количество минут разговора в месяц, не превышающее 676 минут.
- *Максимальное значение* равно 24 229 минутам в месяц.

Полученные описательные статистики могут помочь понять распределение значений столбца 'mou' и выявить возможные выбросы в данных. Например, большое стандартное отклонение и наличие выбросов (максимальное значение в 24,229.434 минут в месяц) могут указывать на наличие аномальных значений в данных, которые могут исказить результаты анализа. Поэтому проверим, сколько строк из всего датасета по городам выше верхней границы макс. значений.

In [ ]:
distr('mou')

Выходит, что мы можем удалить строки с выбросами, так как они не сильно влияют на объем работы с городом (до 10% в худшем случае).

In [ ]:
df_2021 = df_2021[df_2021['mou'] < upper_fence]

mou_after = df_2021.shape[0]

data_lost = (mou_prev - mou_after) / prev * 100
itogo(data_lost)

Теперь повторим шаги и взглянем на __исходящие звонки__. 

In [ ]:
mou_out_prev = df_2021.shape[0]

In [ ]:
df_2021.loc[df_2021['mou_out'].isnull(), 'mou_out'] = 0

In [ ]:
boxplot('mou_out')

In [ ]:
upper_fence, lower_fence = fences('mou_out')

In [ ]:
boxplot('mou_out')

In [ ]:
df_2021[['mou_out']].describe()

In [ ]:
distr('mou_out')

In [ ]:
df_2021[df_2021['mou_out'] > upper_fence].shape[0] / df_2021.shape[0] * 100

В случае с исходящими звонками мы удалим __до 5%__ от общего числа голосового трафика в городе; в общем же - около __3%__ от всего кол-ва записей. 

In [ ]:
df_2021 = df_2021[df_2021['mou_out'] < upper_fence]

mou_out_after = df_2021.shape[0]

data_lost = (mou_out_prev - mou_out_after) / prev * 100
itogo(data_lost)

#### Интернет трафик

Повторим действия из предыдущих шагов. 

In [ ]:
dou_prev = df_2021.shape[0]

In [ ]:
df_2021.loc[df_2021['dou'].isnull(), 'dou'] = 0

In [ ]:
boxplot('dou')

In [ ]:
upper_fence, lower_fence = fences('dou')

In [ ]:
df_2021[['dou']].describe()

До удаления выбросов у нас следующие показатели по интернет трафику:
- _В среднем_ пользователь сети использует 12.4 Гб в месяц;
- Показатель _ст. отклонения_ достаточно высок, чтобы завявить, что данные искажены выбросами. 
- _Минимальное_ значение, равное 0;
- _25%, первый квартиль_, равный 170 Мб;
- _50%, медиана_, равная 4.2 Гб в месяц;
- _75%, третий квартиль_, равный 14.7 Гб в месяц. Что примечательно, показатель третьего квартиля указывает, что среднее потребление интнернет трафика искажено аномальными высокими значениями;
- _Максимальное значение_, равное 1.27 Тб в месяц; возможно кто-то пользуется мобильным интнернетом, как основным. 

Рассмотрим границу максимальных значений. 

In [ ]:
distr('dou')

Если удалять строки по верхней границе, то в некоторых городах мы удалим до 13% от общего кол-ва строк. Разумнее будет рассчитать для каждого города индивидуальную верхнюю границу и рассмотреть таблицу еще раз. 

In [ ]:
# Группируем данные по городам и применяем функцию для расчета квантилей для каждого города
q1 = df_2021.groupby('city')['dou'].apply(lambda x: x.quantile(0.25))
q3 = df_2021.groupby('city')['dou'].apply(lambda x: x.quantile(0.75))

# Рассчитываем межквартильный размах для каждого города
iqr = q3 - q1

cities = df_2021.groupby('city').count()

# Рассчитываем верхнюю границу для каждого города
upper_fence = q3 + 1.5 * iqr

# Группируем исходный датафрейм по городам и считаем количество строк, где значение столбца dou меньше границы максимальных значений
df_filtered = df_2021.groupby('city').apply(lambda x: x[x['dou'] < upper_fence[x.name]].count())

# Рассчитываем процентное соотношение количества строк с меньшим значением dou к общему количеству строк для каждого города
df_filtered['%'] = df_filtered['id_hash'] / cities['id_hash'] * 100

# Создаем новый датафрейм с результатами
df_result = pd.DataFrame({
    'city': cities.index,
    'count_less_uf': df_filtered['id_hash'],
    '%': df_filtered['%']
})
df_result = df_result.reset_index(drop=True)
df_result = df_result.sort_values(by='%')
df_result

Ситуация сильно не поменялась. Тогда опеределим границу максмальных значений самостоятельно. Как мы помним, в среднем абонент потребляет около 12 Гб трафика в месяц, 75% всех абонентов не более 14 Гб, а верхняя граница максимальных значений равна 36.6 Гб. Найдем, сколько границу потребления трафика 95% всех пользователей и установим ее в качестве нашей границы.

In [ ]:
upper_fence = df_2021["dou"].quantile(0.95)
print(f'{upper_fence} Мб – граница потребления 95% всех абонентов в 2021 году')

In [ ]:
distr('dou')

In [ ]:
# # Группируем данные по городам и применяем функцию для расчета квантилей для каждого города
# q1 = df_2021.groupby('city')['dou'].apply(lambda x: x.quantile(0.25))
# q3 = df_2021.groupby('city')['dou'].apply(lambda x: x.quantile(0.75))
# 
# # Рассчитываем межквартильный размах для каждого города
# iqr = q3 - q1
# 
# cities = df_2021.groupby('city').count()
# 
# # Рассчитываем верхнюю границу для каждого города
# upper_fence = q3 + 1.5 * iqr
# 
# # Группируем исходный датафрейм по городам и месяцам, считаем количество строк, где значение столбца dou меньше границы максимальных значений
# df_filtered_month = df_2021.groupby(['city', 'month']).apply(lambda x: x[x['dou'] < upper_fence[x.name[0]]].count())
# df_filtered_month = df_filtered_month.rename(columns={'city':'count', 'month':'month_name_c'})
# df_filtered_month = df_filtered_month.reset_index()
# 
# df_filtered_month['%'] = df_filtered_month['id_hash'] / df_2021.groupby(['city', 'month'])['id_hash'].count().reset_index()['id_hash'] * 100
# 
# # Создаем сводную таблицу, где строки - это города, столбцы - это месяцы, а значения - это количество строк и процентное соотношение для каждого города и каждого месяца
# df_pivot = df_filtered_month.pivot_table(values=['id_hash', '%'], index='city', columns='month', fill_value=0)
# 
# # Объединяем таблицу df_result с таблицей df_pivot по столбцу 'city'
# df_pivot


In [ ]:
df_2021 = df_2021[df_2021['dou'] < upper_fence]

dou_after = df_2021.shape[0]

data_lost = (dou_prev - dou_after) / prev * 100
itogo(data_lost)

Таким образом, мы убрали из таблицы тех пользоватлей, которые использовали аномальный объем трафика.

#### __Вывод__

Итого, мы устранили пропуски за 2021 год:

In [ ]:
df_2021.isnull().sum().sort_values(ascending=False)/df_2021.shape[0]*100

In [ ]:
after = df_2021.shape[0]

data_lost = after / prev * 100 - 100

itogo(data_lost)

* __Возраст__:
> * 174 тыс. абонентов – не удалось определить возраст;
> * 0.74% абонентов было удалено ввиду аномальных значений;
> * 36.2 года – средний возраст абонента;
> * 37 лет – возраст абонента по медиане;
> * Петушки, Покров, Ковров, Лакинск, Горно-Алтайск, Вязники  – лидеры по минимальному среднему возрасту абонента – 35.7-36.2 года, 35-36 по медиане;
> * Курганинск – лидер по максимальному среднему возрасту абонента  – 40.8 года, 40 по медиане;
> * Отфильтровано данных 5.37% от первоначального кол-ва строк.


* __Количество проведенных дней__:
> * Минимальная граница значений до выбросов – 3 дня;
> * Максимальная граница значений до выбросов – 20.5 дней;
> * от _89% до 99.9%_ абонентов 36 городов __проводят в городе до 20.5 дней__;
> * от _51% до 87%_ абонентов 36 городов __проводят в городе от 3 дней__ (включительно); 
> * Отфильтровано данных 1.62% от первоначального кол-ва строк.

* __Домашний регион__:
> * Топ-5 регионов по кол-ву туристов: Москва (633 тыс.), Нижегородская обл. (224 тыс.), Ростовская обл. (211 тыс.), Санкт-Петербург (195 тыс.) и Самарская обл. (160 тыс.)
> * Топ-5 регионов по наименьшему кол-ву туристов: Еврейская автономная обл. (859), Магаданская обл. (422), Республика Алтай (305), Республика Адыгея (289), Чукотский автономный округ. (130)
> * 14 тыс. строк – с регионом "-1". (в проуессе получения ответа от менторов)

* __Туристический регион__:
> * 78% всех полей с регионом "Краснодарский край";
> * 0.7% всех полей с регионом "Республика Карелия";
> * Аномалии отсутствуют;

* __Пол абонента__:
> * 46.9% всех абонентов – мужчины;
> * 42.4% всех абонентов – женщмны;
> * 10.7% пол не определен
> * Аномалии отсутствуют;


* __Голосовой трафик общий__:
> * Минимальная граница значений до выбросов – 1575 минуты;
> * Максимальная граница значений до выбросов – 73 минуты;
> * 479 минут – в среднем потребление голосового трафика в месяц;
> * 295 минут – потребление голсового трафика в месяц по медиане;
> * Отфильтровано данных 5.11% от первоначального кол-ва строк.

* __Голосовой трафик исходящий__:
> * Минимальная граница значений до выбросов – 739 минуты;
> * Максимальная граница значений до выбросов – 21 минуты;
> * 200 минут – в среднем потребление голосового трафика в месяц;
> * 126 минут – потребление голсового трафика в месяц по медиане;
> * Отфильтровано данных 2.96% от первоначального кол-ва строк.

* __Интернет трафик__:
> * Минимальная граница значений до выбросов – 159 МБ или 0.16 ГБ;
> * Максимальная граница значений до выбросов – 36.5 ГБ;
> * 12.3 ГБ – в среднем потребление интернет трафика в месяц;
> * 4.2 ГБ – потребление интернет трафика в месяц по медиане;
> * 95-ый перцентиль – 51.8 ГБ как граница фильтрации абонентов;
> * Отфильтровано данных 4.56% от первоначального кол-ва строк.


### 2022 год

#### Возраст

In [ ]:
def boxplot(field):
    plt.boxplot(df_2022[field], showcaps=True)

    # добавляем заголовок и метки осей
    plt.title(f'{field} Distribution')
    plt.ylabel(f'{field}')
    
    # выводим boxplot на экран
    plt.show()

In [ ]:
boxplot('age')

In [ ]:
df_2022[df_2022['age'] < 0].shape[0]

In [ ]:
df_2022[df_2022['age'] == 0].shape[0]

Почти в 3 раза больше за 2022 год стало абонентов, чей возраст определить не удалось. Это может повлиять на средний возраст абонентов и медиану. 

In [ ]:
df_2022['age'] = df_2022['age'].apply(lambda x: 0 if x > 99 or x < 0 else x)

In [ ]:
boxplot('age')

In [ ]:
def fences(field):
    q1 = df_2022[field].quantile(0.25)
    q3 = df_2022[field].quantile(0.75)
    iqr = q3 - q1
    upper_fence = q3 + 1.5 * iqr
    lower_fence = q1
    
    print(f'Граница максимальных значений {field}:', upper_fence)
    print(f'Граница минимальных значений {field}:', lower_fence)
    return upper_fence, lower_fence

In [ ]:
def distr(field, ul=1):
    if ul == 1:
        cities = df_2022.groupby('city').count()
        
        # Группируем исходный датафрейм по городам и считаем количество строк, где значение столбца field меньше границы максимальных значений
        df_filtered = df_2022[df_2022[field] < upper_fence].groupby('city').count()
        
        # Рассчитываем процентное соотношение количества строк с меньшим значением field к общему количеству строк для каждого города
        df_filtered['%'] = df_filtered['id_hash'] / cities['id_hash'] * 100
        
        # Создаем новый датафрейм с результатами
        df_result = pd.DataFrame({
            'city': cities.index,
            'count_less_uf': df_filtered['id_hash'],
            '%': df_filtered['%']
        })
        
        df_result = df_result.reset_index(drop=True)
        df_result = df_result.sort_values(by='%')
        return df_result
    else:
        cities = df_2022.groupby('city').count()
        
        # Группируем исходный датафрейм по городам и считаем количество строк, где значение столбца field меньше границы максимальных значений
        df_filtered = df_2022[df_2022[field] > lower_fence].groupby('city').count()
        
        # Рассчитываем процентное соотношение количества строк с меньшим значением field к общему количеству строк для каждого города
        df_filtered['%'] = df_filtered['id_hash'] / cities['id_hash'] * 100
        
        # Создаем новый датафрейм с результатами
        df_result = pd.DataFrame({
            'city': cities.index,
            'count_more_lf': df_filtered['id_hash'],
            '%': df_filtered['%']
        })
        
        df_result = df_result.reset_index(drop=True)
        df_result = df_result.sort_values(by='%')
        return df_result
        

In [ ]:
upper_fence, lower_fence = fences('age')

In [ ]:
distr('age')

In [ ]:
distr('age', 2)

Что можем отметить, так это меньшие значения максимальной (с 65 до 63.5) и минимальной границ (с 30 до 26) для 2022 года в сравнении с 2021, что говорит, что возраст туристов снизился и чаще более молодые люди начали путешествовать.

Если Курганинск чаще других городов (но совсем незгначительно) посещают люди старше 63.5 лет (коих менее 4% от общего кол-ва за год), то такие города, как Гороховец (34.8%), Покров (33.5%), Ковров (32%), Вязники	(30.68%), Лакинск (30.64%), Меленки	(30%) посещают чаще других туристы до 26 лет, судя по предварительным данным. Но помним момент с бóльшим кол-ом абонентов с 0 значением в графе с их возрастом. Отфильтруем данные по 65 лет, как в таблице за 2021 год.

In [ ]:
lower_fence = 1

In [ ]:
distr('age', 2)

в 33 из 36 городов доля абонентов с 0 в графе `age` составило более 10%, но может сказаться на расчетах уникальных пользователей в дальнейшем. Поэтому было принято решение оставить таких пользоваталей в основном дф, отдельно рассчитав параметры возрастов в разных городов без 0 значений. 

In [ ]:
age_prev = df_2022.shape[0]

df_2022 = df_2022[df_2022['age'] < 65]

age_after = df_2022.shape[0]

data_lost = (age_prev - age_after) / prev * 100

def itogo(field):
    return print(f'Итого после предобработки данных мы потеряли {field:.2f}%')

itogo(data_lost)


Для устранения аномалий с возрастами абонентов мы пожертвовали более чем в 2 раза больше данных, чем за 2021 год.

In [ ]:
df_2022_no_age = df_2022[df_2022['age'] > 0]

In [ ]:
df_2022_no_age.groupby('city')['age'].agg(['mean', 'median', q25, q75, 'min', 'max']).sort_values(by=['mean', 'median'])

Во многих городах абоненты в среднем и по медиане стале моложе. 

#### Количество проведенных дней

In [ ]:
days_prev = df_2022.shape[0]

In [ ]:
boxplot('days')

In [ ]:
df_2022[df_2022['days'] < 5].shape[0]

Так же, как и за 2021 год в таблице около 1.2 млн. пользователей, которые были в городе до 5 дней. 

In [ ]:
less_5 = (df_2022
          .pivot_table(index='id_hash',
                       columns='month',
                       values='days',
                       aggfunc='sum',
                       margins=True)
          .sort_values(by='All',
                       ascending=False))

less_5 = less_5.query('All >= 5')
users = less_5.index

In [ ]:
df_2022 = df_2022[df_2022['id_hash'].isin(users)]

In [ ]:
fences('days')

In [ ]:
upper_fence, lower_fence = (22, 4)

In [ ]:
distr('days')

In [ ]:
distr('days', 2)

Похожая с 2021 годом картина. Дни распределены примерно одинаоково. 

In [ ]:
df_2022 = df_2022[df_2022['days'] < 22]

days_after = df_2022.shape[0]

data_lost = (days_prev - days_after) / prev * 100 
itogo(data_lost)

In [ ]:
df_2022[['days']].describe()

#### Домашний регион

In [ ]:
df_2022.groupby('home_region').agg({'id_hash':'count'})

"Отрицательных" регионов меньше. 

In [ ]:
home_2022 = df_2022.groupby('home_region').agg({'id_hash':'count'}).sort_values(by='id_hash', ascending=False)

In [ ]:
home_2022['%'] = home_2022['id_hash'] / home_2022['id_hash'].sum() * 100
home_2022

Кол-во туристов из Москвы стало больше на 30 с небольшим тысяч человек, из Ростовской области туристов почти не изменилось, но туристы оттуда теперь на 2-м месте по общему числу поездок; из Нижегородской области туристов стало меньше на 30 тыс. человек или около -0.9% от общего числа. Из СПб туристов практически не поменялось, из Самарской области туристов стало меньше на примерно 7-8 тыс. человек. 

В топе непутешествующих все те же 4 региона – Еврейская АТО, Магаданская обл., Алтай, Чукотский АТО плюс Республика Тыва. 

#### Туристический регион

In [ ]:
region_counts = df_2022['tourists_region'].value_counts()

# Вычисляем общее количество строк в датасете
total_rows = len(df_2022)

# Строим таблицу, где для каждого региона показано процентное соотношение количества абонентов от общего количества строк в датасете
result = pd.concat([region_counts, region_counts / total_rows * 100], axis=1)
result.columns = ['count', '%']

result

Поток в Краснодарский край и Владимирскую обл. увеличился, хоть и незначительно, тогда как в другие реигионы он упал. 

In [ ]:
# Добавляем новый столбец с наименованием региона
df_2022['tourist_region_name'] = df_2022['tourists_region'].apply(get_region_name)

#### Пол

In [ ]:
age_counts = df_2022['gender'].value_counts()

# Вычисляем общее количество строк в датасете
total_rows = len(df_2022)

# Строим таблицу, где для каждого региона показано процентное соотношение количества абонентов от общего количества строк в датасете
result = pd.concat([age_counts, age_counts / total_rows * 100], axis=1)
result.columns = ['count', '%']

result

Доля "бесполых" увеличилась на почти вдвое, женщин стало меньше, как и мужчин. Модель предсказания стала хуже определять пол. 

#### Голосовой трафик и голосовой исходящий трафик

In [ ]:
mou_prev = df_2022.shape[0]

In [ ]:
df_2022.loc[df_2022['mou'].isnull(), 'mou'] = 0

In [ ]:
boxplot('mou')

In [ ]:
upper_fence, lower_fence = fences('mou')

In [ ]:
df_2022[['mou']].describe()

In [ ]:
distr('mou')

In [ ]:
df_2022 = df_2022[df_2022['mou'] < upper_fence]

mou_after = df_2022.shape[0]

data_lost = (mou_prev - mou_after) / prev * 100
itogo(data_lost)

__Голосовой исходящий трафик__

In [ ]:
mou_out_prev = df_2022.shape[0]

In [ ]:
df_2022.loc[df_2022['mou_out'].isnull(), 'mou_out'] = 0

In [ ]:
boxplot('mou_out')

In [ ]:
upper_fence, lower_fence = fences('mou_out')

In [ ]:
boxplot('mou_out')

In [ ]:
df_2022[['mou_out']].describe()

In [ ]:
distr('mou_out')

In [ ]:
df_2022[df_2022['mou_out'] > upper_fence].shape[0] / df_2022.shape[0] * 100

In [ ]:
df_2022 = df_2022[df_2022['mou_out'] < upper_fence]

mou_out_after = df_2022.shape[0]

data_lost = (mou_out_prev - mou_out_after) / prev * 100
itogo(data_lost)

#### Интернет трафик

In [ ]:
dou_prev = df_2022.shape[0]

In [ ]:
df_2022.loc[df_2022['dou'].isnull(), 'dou'] = 0

In [ ]:
boxplot('dou')

In [ ]:
upper_fence, lower_fence = fences('dou')

In [ ]:
df_2022[['dou']].describe()

- В среднем пользователь использует 13.8 Гб в месяц, что является большим значением, чем в предыдущей таблице за 2021 год. 
- Стандартное отклонение также выше, что указывает на более высокую вариативность данных. 
- Минимальное значение также равно 0, а максимальное значение равно 1.55 Тб в месяц, что является большим значением, чем в предыдущей таблице.

Сравнивая показатели двух таблиц, можно сделать вывод, что пользователи в 2022 году в среднем используют больше интернет трафика, чем пользователи за 2021 год. Однако, в данной таблице также можно наблюдать выбросы в виде аномально высоких значений, которые могут искажать среднее значение и делать его нерепрезентативным. Поэтому перед анализом данных следует удалять выбросы, чтобы получить более точные результаты.

In [ ]:
distr('dou')

In [ ]:
upper_fence = df_2022["dou"].quantile(0.95)
print(f'{upper_fence} Мб – граница потребления 95% всех абонентов в 2022 году')

In [ ]:
distr('dou')

In [ ]:
df_2022 = df_2022[df_2022['dou'] < upper_fence]

dou_after = df_2022.shape[0]

data_lost = (dou_prev - dou_after) / prev * 100
itogo(data_lost)

#### **Вывод**

In [ ]:
df_2022.isnull().sum().sort_values(ascending=False)/df_2022.shape[0]*100

In [ ]:
after = df_2022.shape[0]

data_lost = after / prev * 100 - 100

itogo(data_lost)

# Обзор данных

In [ ]:
df_2021.shape[0]

In [ ]:
df_2022.shape[0]

## Временные ряды

In [ ]:
# Создаем список городов
eng_cities = ['Abinsk', 'Alexandrov', 'Armavir', 'Baikalsk', 'Belorechensk',
       'Vyazniki', 'Gorno-Altaisk', 'Gorodishchi', 'Gorokhovets',
       'Hot Key', 'Gulkevichi', 'Yeisk', 'Kovrov', 'Kolchugino',
       'Korenovsk', 'Kropotkin', 'Krymsk', 'Kurganinsk', 'Labinsk',
       'Lakinsk', 'Melenki', 'Murom', 'Novokubansk', 'Petushki', 'Pokrov',
       'Sirius', 'Slavyansk-on-Kuban', 'Sortavala', 'Strunino', 'Temryuk',
       'Timashevsk', 'Tikhoretsk', 'Tuapse', 'Ust-Labinsk', 'Khadyzhensk',
       'Yuriev-Polsky']

cities = ['Абинск', 'Александров', 'Армавир', 'Байкальск', 'Белореченск',
       'Вязники', 'Горно-Алтайск', 'Городищи', 'Гороховец',
       'Горячий Ключ', 'Гулькевичи', 'Ейск', 'Ковров', 'Кольчугино',
       'Кореновск', 'Кропоткин', 'Крымск', 'Курганинск', 'Лабинск',
       'Лакинск', 'Меленки', 'Муром', 'Новокубанск', 'Петушки', 'Покров',
       'Сириус', 'Славянск-на-Кубани', 'Сортавала', 'Струнино', 'Темрюк',
       'Тимашевск', 'Тихорецк', 'Туапсе', 'Усть-Лабинск', 'Хадыженск',
       'Юрьев-Польский']

# Создаем цикл, который будет проходить по каждому городу и создавать переменную с данными из двух датафреймов
for city, r_city in zip(eng_cities, cities):
    globals()[city] = pd.concat([df_2021[df_2021['city'] == r_city], df_2022[df_2022['city'] == r_city]])



In [ ]:
Lakinsk.head()

## Уникальные абоненты

### Ежегодно

In [ ]:
def abon_unique(df, year):
        table = (df
                 .groupby(['month', 'month_name'])
                 .agg({'id_hash':'nunique'})
                 .sort_values(by='month')
                 .reset_index())
        table['%'] = table['id_hash'] / table['id_hash'].sum() * 100
        table['%'] = table['%'].round(2)
        
        fig = px.bar(table,
               x='month_name', 
               y='id_hash',
               text_auto=True,
               title=f'Количество абонентов услуг ежемесячно в {year}',
               text='%')
        fig.show()

In [ ]:
abon_unique(df_2021, 2021)

In [ ]:
abon_unique(df_2022, 2022)

In [ ]:
# группировка данных по городам и подсчет количества уникальных пользователей в каждом году
df_2021_grouped = df_2021.groupby('city')['id_hash'].nunique().reset_index()
df_2022_grouped = df_2022.groupby('city')['id_hash'].nunique().reset_index()

# объединение данных по городам в одну таблицу
merged_df = pd.merge(df_2021_grouped, df_2022_grouped, on='city', suffixes=('_2021', '_2022'))

# добавление столбца с процентным изменением количества пользователей
merged_df['%'] = (merged_df['id_hash_2022'] - merged_df['id_hash_2021']) / merged_df['id_hash_2021'] * 100
merged_df['%'] = merged_df['%'].round(2)

# переименование столбцов для более понятной интерпретации
merged_df = merged_df.rename(columns={'id_hash_2021': '2021', 'id_hash_2022': '2022'})

merged_df['abs_diff'] = merged_df['2022'] - merged_df['2021']

# вывод итоговой таблицы
merged_df = merged_df.sort_values(by='abs_diff', ascending=False).reset_index(drop=True)

In [ ]:
merged_df

In [ ]:
import plotly.express as px

fig = px.bar(merged_df,
             x='abs_diff',
             y='city',
             orientation='h',
             height=1000, 
             width=900, 
             text='%',
             title='Абсолютное и процентное изменение количества уникальных пользоваталей в 2021 и 2022 году',
             labels={'abs_diff':'уникальных абонентов'})
fig.update_traces(texttemplate='%{text}%')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


### Ежемесячно

In [ ]:
# объединение датасетов по строкам
df = pd.concat([df_2021, df_2022], ignore_index=True)

In [ ]:
cities = df.city.unique().tolist()

In [ ]:
for city in cities:
    # создание сводной таблицы с количеством уникальных id по месяцам и годам
    pivot_table = pd.pivot_table(df[df['city'] == city], index='month', columns='year', values='id_hash', aggfunc=pd.Series.nunique)

    # добавление столбца с процентным изменением количества абонентов
    pivot_table['%'] = (pivot_table[2022] - pivot_table[2021]) / pivot_table[2021] * 100

    # вывод таблицы
    pivot_table['abs_diff'] = pivot_table[2022] - pivot_table[2021]
    pivot_table = pivot_table.reset_index(drop=False)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=[calendar.month_name[i] for i in pivot_table['month']], y=pivot_table[2021], name='2021'))
    fig.add_trace(go.Bar(x=[calendar.month_name[i] for i in pivot_table['month']], y=pivot_table[2022], name='2022'))
    fig.add_trace(go.Scatter(x=[calendar.month_name[i] for i in pivot_table['month']], y=pivot_table[2021].rolling(window=3).mean(),
                             mode='lines', name='Тренд 2021', line_color='blue'))
    fig.add_trace(go.Scatter(x=[calendar.month_name[i] for i in pivot_table['month']], y=pivot_table[2022].rolling(window=3).mean(),
                             mode='lines', name='Тренд 2022', line_color='red'))
    fig.update_layout(title=f'Ежемесячное количество абонентов в {city} в 2021 и 2022 году',
                      xaxis_title='Месяц', yaxis_title='Количество абонентов')
    fig.show()



## Функции

In [ ]:
def corr_whole(data, year):
    corr_matrix = data.corr()
    # создаем тепловую карту на основе матрицы корреляции
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    
    # устанавливаем заголовок графика
    plt.title(f'Матрица корреляции для всех городов {year} года')
    
    # выводим график на экран
    return plt.show()

In [ ]:
def corr_cities(data, year):
    for city in cities:    
        df = data[data['city'] == city]
        corr_matrix = df.corr()
        # создаем тепловую карту на основе матрицы корреляции
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        
        # устанавливаем заголовок графика
        plt.title(f'Матрица корреляции для г. {city} в 2021 году')
        
        # выводим график на экран
        plt.show()

In [ ]:
def abon_unique_city(df, year):
    for city in cities:
        table = (df[df['city'] == city]
                 .groupby(['month', 'month_name'])
                 .agg({'id_hash':'nunique'})
                 .sort_values(by='month')
                 .reset_index())
        table['%'] = table['id_hash'] / table['id_hash'].sum() * 100
        table['%'] = table['%'].round(2)
        
        fig = px.bar(table,
               x='month_name', 
               y='id_hash',
               text_auto=True,
               title=f'Количество абонентов услуг ежемесячно в {city} в {year}',
               text='%')
        fig.show()

## Матрица корреляции

### 2021 год

#### Корреляция по всем городам

In [ ]:
corr_whole(df_2021, 2021)

#### Корреляция отдельно по городам

In [ ]:
def corr_cities(data, year):
    for city in cities:    
        df = data[data['city'] == city]
        corr_matrix = df.corr()
        # создаем тепловую карту на основе матрицы корреляции
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        
        # устанавливаем заголовок графика
        plt.title(f'Матрица корреляции для г. {city} в 2021 году')
        
        # выводим график на экран
        plt.show()

In [ ]:
corr_cities(df_2021, 2021)

### 2022 год

#### Корреляция по всем городам

In [ ]:
corr_whole(df_2022, 2022)

#### Корреляция отдельно по городам

In [ ]:
corr_cities(df_2022, 2022)

## Количество уникальных абонентов ежемесячно за 2021

In [ ]:
abon_unique(df_2021, 2021)